# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7d54d9dbb0>
├── 'a' --> tensor([[-0.1275,  2.1602, -1.6760],
│                   [-0.3528,  0.0810, -0.5175]])
└── 'x' --> <FastTreeValue 0x7f7d54d9d100>
    └── 'c' --> tensor([[-0.4122,  0.3281,  0.5373,  1.0028],
                        [-0.8598,  2.5058, -0.6549, -1.9361],
                        [-1.7551,  0.3537,  0.3212, -0.6408]])

In [4]:
t.a

tensor([[-0.1275,  2.1602, -1.6760],
        [-0.3528,  0.0810, -0.5175]])

In [5]:
%timeit t.a

88.9 ns ± 0.71 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7d54d9dbb0>
├── 'a' --> tensor([[-0.5290, -0.2250, -1.2578],
│                   [ 1.2898,  0.9858, -0.2898]])
└── 'x' --> <FastTreeValue 0x7f7d54d9d100>
    └── 'c' --> tensor([[-0.4122,  0.3281,  0.5373,  1.0028],
                        [-0.8598,  2.5058, -0.6549, -1.9361],
                        [-1.7551,  0.3537,  0.3212, -0.6408]])

In [7]:
%timeit t.a = new_value

93.3 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1275,  2.1602, -1.6760],
               [-0.3528,  0.0810, -0.5175]]),
    x: Batch(
           c: tensor([[-0.4122,  0.3281,  0.5373,  1.0028],
                      [-0.8598,  2.5058, -0.6549, -1.9361],
                      [-1.7551,  0.3537,  0.3212, -0.6408]]),
       ),
)

In [10]:
b.a

tensor([[-0.1275,  2.1602, -1.6760],
        [-0.3528,  0.0810, -0.5175]])

In [11]:
%timeit b.a

80.8 ns ± 0.318 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3095, -0.2105, -0.0669],
               [-0.6022,  1.5463,  0.0598]]),
    x: Batch(
           c: tensor([[-0.4122,  0.3281,  0.5373,  1.0028],
                      [-0.8598,  2.5058, -0.6549, -1.9361],
                      [-1.7551,  0.3537,  0.3212, -0.6408]]),
       ),
)

In [13]:
%timeit b.a = new_value

660 ns ± 4.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.3 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

175 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

176 µs ± 3.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7d54d98e50>
├── 'a' --> tensor([[[-0.1275,  2.1602, -1.6760],
│                    [-0.3528,  0.0810, -0.5175]],
│           
│                   [[-0.1275,  2.1602, -1.6760],
│                    [-0.3528,  0.0810, -0.5175]],
│           
│                   [[-0.1275,  2.1602, -1.6760],
│                    [-0.3528,  0.0810, -0.5175]],
│           
│                   [[-0.1275,  2.1602, -1.6760],
│                    [-0.3528,  0.0810, -0.5175]],
│           
│                   [[-0.1275,  2.1602, -1.6760],
│                    [-0.3528,  0.0810, -0.5175]],
│           
│                   [[-0.1275,  2.1602, -1.6760],
│                    [-0.3528,  0.0810, -0.5175]],
│           
│                   [[-0.1275,  2.1602, -1.6760],
│                    [-0.3528,  0.0810, -0.5175]],
│           
│                   [[-0.1275,  2.1602, -1.6760],
│                    [-0.3528,  0.0810, -0.5175]]])
└── 'x' --> <FastTreeValue 0x7f7d2c2bbfd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.5 µs ± 91.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7d2c2bbf70>
├── 'a' --> tensor([[-0.1275,  2.1602, -1.6760],
│                   [-0.3528,  0.0810, -0.5175],
│                   [-0.1275,  2.1602, -1.6760],
│                   [-0.3528,  0.0810, -0.5175],
│                   [-0.1275,  2.1602, -1.6760],
│                   [-0.3528,  0.0810, -0.5175],
│                   [-0.1275,  2.1602, -1.6760],
│                   [-0.3528,  0.0810, -0.5175],
│                   [-0.1275,  2.1602, -1.6760],
│                   [-0.3528,  0.0810, -0.5175],
│                   [-0.1275,  2.1602, -1.6760],
│                   [-0.3528,  0.0810, -0.5175],
│                   [-0.1275,  2.1602, -1.6760],
│                   [-0.3528,  0.0810, -0.5175],
│                   [-0.1275,  2.1602, -1.6760],
│                   [-0.3528,  0.0810, -0.5175]])
└── 'x' --> <FastTreeValue 0x7f7d54daff40>
    └── 'c' --> tensor([[-0.4122,  0.3281,  0.5373,  1.0028],
                        [-0.8598,  2.5058, -0.6549, -1.9361],
                 

In [23]:
%timeit t_cat(trees)

36.7 µs ± 528 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.3 µs ± 790 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1275,  2.1602, -1.6760],
                [-0.3528,  0.0810, -0.5175]],
       
               [[-0.1275,  2.1602, -1.6760],
                [-0.3528,  0.0810, -0.5175]],
       
               [[-0.1275,  2.1602, -1.6760],
                [-0.3528,  0.0810, -0.5175]],
       
               [[-0.1275,  2.1602, -1.6760],
                [-0.3528,  0.0810, -0.5175]],
       
               [[-0.1275,  2.1602, -1.6760],
                [-0.3528,  0.0810, -0.5175]],
       
               [[-0.1275,  2.1602, -1.6760],
                [-0.3528,  0.0810, -0.5175]],
       
               [[-0.1275,  2.1602, -1.6760],
                [-0.3528,  0.0810, -0.5175]],
       
               [[-0.1275,  2.1602, -1.6760],
                [-0.3528,  0.0810, -0.5175]]]),
    x: Batch(
           c: tensor([[[-0.4122,  0.3281,  0.5373,  1.0028],
                       [-0.8598,  2.5058, -0.6549, -1.9361],
                       [-1.7551,  0.3537,  0.3212, -0.6408]],
         

In [26]:
%timeit Batch.stack(batches)

97.4 µs ± 934 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1275,  2.1602, -1.6760],
               [-0.3528,  0.0810, -0.5175],
               [-0.1275,  2.1602, -1.6760],
               [-0.3528,  0.0810, -0.5175],
               [-0.1275,  2.1602, -1.6760],
               [-0.3528,  0.0810, -0.5175],
               [-0.1275,  2.1602, -1.6760],
               [-0.3528,  0.0810, -0.5175],
               [-0.1275,  2.1602, -1.6760],
               [-0.3528,  0.0810, -0.5175],
               [-0.1275,  2.1602, -1.6760],
               [-0.3528,  0.0810, -0.5175],
               [-0.1275,  2.1602, -1.6760],
               [-0.3528,  0.0810, -0.5175],
               [-0.1275,  2.1602, -1.6760],
               [-0.3528,  0.0810, -0.5175]]),
    x: Batch(
           c: tensor([[-0.4122,  0.3281,  0.5373,  1.0028],
                      [-0.8598,  2.5058, -0.6549, -1.9361],
                      [-1.7551,  0.3537,  0.3212, -0.6408],
                      [-0.4122,  0.3281,  0.5373,  1.0028],
                      [-0.8598,  

In [28]:
%timeit Batch.cat(batches)

178 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

453 µs ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
